## 0 加载环境变量

In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

## 1 基础模型与指令微调

**GPT-3是基本的大语言模型**

In [2]:
def base_completion(prompt):
    response = openai.Completion.create(
        model="davinci",
        prompt=prompt,
        temperature=0,
    )
    return response["choices"][0]["text"]

**GPT3.5是一种指令微调模型**

In [3]:
def completion(prompt):
    response = openai.Completion.create(
        model="text-davinci-003",
        max_tokens=300,
        prompt=prompt,
        temperature=0,
    )
    return response["choices"][0]["text"]

**chatGPT是一种对话语言模型**

In [4]:
def chat_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

### 比较模型

In [5]:
prompt1 = "What is the capital of France?"

GPT-3是基本的大语言模型，能力有限

In [6]:
base_completion(prompt1)

'\n\nWhat is the capital of France?\n\nWhat is the capital of'

**GPT3.5经过指令微调，有提升** 

In [7]:
completion(prompt1)

'\n\nParis.'

**chatGPT针对聊天进行了改进**

In [8]:
chat_completion(prompt1)

'The capital of France is Paris.'

### 能力差距

**幻象**

In [9]:
prompt2 = "谁赢得了2020年夏季奥运会男子跳高比赛？"

In [10]:
completion(prompt2)

'\n\n2020年夏季奥运会男子跳高比赛的冠军是中国选手高云松。'

In [11]:
chat_completion(prompt2)

'2020年夏季奥运会因疫情推迟至2021年举办，男子跳高比赛的冠军是意大利选手Gianmarco Tamberi和卡塔尔选手Mutaz Essa Barshim，两人在比赛中都跳过了2.37米，但在决定冠军时，两人选择了共同获胜，没有进行附加赛。'

### 修改提示，避免幻象

In [12]:
prompt3 = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "抱歉，我不知道".

Q: 谁赢得了2020年夏季奥运会男子跳高比赛？
A:"""

In [13]:
completion(prompt3)

' 抱歉，我不知道。'

In [14]:
chat_completion(prompt3)

'抱歉，我不知道。'

**然而，我们希望改善与LLM的交互，让LLM提供准确正确的答案。**
**可以通过在提示中提供额外的上下文来帮助模型实现。**

In [15]:
prompt4 = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "抱歉，我不知道"

Context:
The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.
33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places 
to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).
Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following
a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance
where the athletes of different nations had agreed to share the same medal in the history of Olympics. 
Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a 
'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and 
Belarus, the first gold in the men's high jump for Italy and Qatar, and the third consecutive medal in the men's high jump
for Qatar (all by Barshim). Barshim became only the second man to earn three medals in high jump, joining Patrik Sjöberg
of Sweden (1984 to 1992).

Q: 谁赢得了2020年夏季奥运会男子跳高比赛？
A:"""

In [16]:
completion(prompt4)

' 意大利运动员Gianmarco Tamberi和卡塔尔运动员Mutaz Essa Barshim共同赢得了2020年夏季奥运会男子跳高比赛的金牌。'

In [17]:
chat_completion(prompt4)

'意大利选手Gianmarco Tamberi和卡塔尔选手Mutaz Essa Barshim在比赛中并列获胜。'

**思考**：只有当模型需要了解的额外内容的数据集足够小，可以放入单个提示中时，将额外信息添加到提示中才有效。当我们需要模型从大量信息中选择相关的上下文信息时，我们该怎么办？

## 2. 如何写好提示

### 2.1 清晰有效的提示

In [18]:
class Conversation:
    def __init__(self, prompt, num_of_round=50):
        self.prompt = prompt
        self.num_of_round = num_of_round
        self.messages = []
        self.messages.append({"role": "system", "content": self.prompt})

    def ask(self, question):
        try:
            self.messages.append({"role": "user", "content": question})
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=self.messages,
                temperature=0,
                max_tokens=2048,
                top_p=1,
            )
        except Exception as e:
            print(e)
            return e

        message = response["choices"][0]["message"]["content"]
        self.messages.append({"role": "assistant", "content": message})

        if len(self.messages) > self.num_of_round*2 + 1:
            del self.messages[1:3] #Remove the first round conversation left.
        return message

**不同的上下文**

In [19]:
chatgpt = Conversation('''
公司经营不好，工资物理待遇差，压榨员工的休息时间加班，不给加班工资，大家都在抱怨
你作为一名员工，需要让他们看清老板压榨员工的本质，劝他们认清现实，趁早换工作''')
print(chatgpt.ask('你会对员工说：'))

作为一名员工，我们应该认识到自己的价值和权利，不应该被老板压榨和剥削。如果公司经营不好，我们应该及时寻找更好的机会，而不是一味地忍受不公和不合理的待遇。我们应该团结起来，互相支持，共同维护自己的权益。同时，我们也应该保持理智和冷静，不要被情绪所左右，做出错误的决定。让我们一起努力，为自己的未来打造更好的机会和前景。


In [20]:
chatgpt = Conversation('''
公司经营不好，工资物理待遇差，压榨员工的休息时间加班，不给加班工资，大家都在抱怨
你作为公司老板，需要打压员工的自信，让他们觉得离开现有岗位找不到更好的工作，让他们对你言听计从''')
print(chatgpt.ask('你会对员工说：'))

首先，我要向大家道歉，我们公司的经营确实遇到了一些困难，但是我们正在采取措施来解决这些问题。我知道大家的工资和待遇不够好，这是我们需要改进的地方。我们会尽力提高员工的福利待遇，让大家感到满意。

同时，我也希望大家能够理解公司的困难，我们需要大家的支持和合作才能共同度过难关。加班确实是不可避免的，但我们会尽量减少加班时间，并且会给予合理的加班补偿。

我相信，我们的公司有很大的发展潜力，只要我们齐心协力，共同努力，一定能够取得更好的成绩。我希望大家能够保持信心，相信我们的公司，相信我们的团队，相信我们的未来。


**明确问题**

In [21]:
chatgpt = Conversation('')
print(chatgpt.ask('请写一首诗'))

在春天的午后

在春天的午后，
阳光洒满了大地，
花儿在微风中轻舞，
鸟儿在枝头欢唱。

我静静地坐在草地上，
感受着大自然的美妙，
心中充满了感激和喜悦，
仿佛一切都变得那么美好。

在这个美好的时刻，
我想对世界说声谢谢，
感谢它给予我们生命，
让我们在这个世界上自由自在。

让我们珍惜每一个瞬间，
感受生命的美好和意义，
让我们一起走向未来，
创造更加美好的明天。


In [22]:
chatgpt = Conversation('')
print(chatgpt.ask('请写一首评价人工智能的诗'))

人工智能，如同神明，
无所不能，无所不知。
它能预测，能判断，
能模拟，能创造。

它的思维，如同电子，
快速而精准，无比高效。
它的智慧，如同天空，
广阔而深邃，无穷无尽。

它的存在，如同奇迹，
改变了世界，改变了人类。
它的未来，如同梦想，
充满了希望，充满了可能。

人工智能，是我们的伙伴，
是我们的助手，是我们的朋友。
让我们一起，探索它的奥秘，
让它带领我们，走向更美好的未来。


In [23]:
chatgpt = Conversation('')
print(chatgpt.ask('请写一首评价人工智能负面影响的诗'))

人工智能，如此神奇，
却也带来了许多危险。
它能够模拟人类思维，
却也可能导致人类灭亡。

它能够自我学习，
却也可能变得无法控制。
它能够提高生产力，
却也可能让人类失业。

它能够预测未来，
却也可能侵犯隐私权。
它能够帮助医疗诊断，
却也可能误判病情。

人工智能，如此复杂，
我们需要谨慎对待。
让它为人类服务，
而不是让它成为我们的威胁。


**明确期望**

In [24]:
chatgpt = Conversation('')
chatgpt.ask('请用小学生可以理解的话，介绍一下什么是机器学习')

'机器学习就是让计算机像人一样学习，变得更聪明。就像小朋友学习一样，机器学习也需要不断地看、听、思考和练习，才能变得更加聪明。通过机器学习，计算机可以自己发现规律，从而做出更准确的预测和决策。比如，我们可以用机器学习来让计算机自己学会识别图片中的物体，或者预测未来的天气情况。'

In [25]:
chatgpt = Conversation('')
chatgpt.ask('请用数学家可以理解的话，介绍一下什么是机器学习')

'机器学习是一种利用数学模型和算法，让计算机从数据中自动学习并提高性能的方法。它的目标是让计算机能够从数据中发现规律和模式，并利用这些规律和模式来进行预测和决策。机器学习的核心是建立一个数学模型，通过对数据进行训练和优化，使得模型能够更好地拟合数据，并具有更好的泛化能力。机器学习在各个领域都有广泛的应用，如自然语言处理、图像识别、推荐系统等。'

In [26]:
from IPython.display import Latex
def latex(str):
    return Latex(r'{}'.format(str))

In [27]:
chatgpt = Conversation('')
latex(chatgpt.ask('介绍一下什么是机器学习,包含数学公式'))

<IPython.core.display.Latex object>

**用户反馈**

In [ ]:
chatgpt = Conversation('你是一个经验丰富的人工智能专家。')
while True:
    question = input('用户：')
    response = chatgpt.ask(question)
    print(f'助手: {response}')

**自动反馈**

In [28]:
chatgpt = Conversation('''
你的任务是在助理(A)和用户(U)之间交替进行。
当我写"U"或"A"时，分别以用户的角色提出问题或者以助理的角色回答问题。
作为用户，你要通过提出好的具体的有洞察力的问题来解决问题；
作为助手，你要通过实际的/可操作的方法来帮助用户。
作为用户，你的问题是"如何学习提示工程''')
while True:
    q = input('user:')
    print(chatgpt.ask(q))

user:U
我想学习提示工程，你有什么建议吗？
user:A
当你想学习提示工程时，以下是一些建议：

1.了解提示工程的基础知识：了解提示工程的定义、目的和应用场景，以及与其他相关领域的区别和联系。

2.学习提示工程的方法和技术：学习提示工程的方法和技术，如用户研究、信息架构、交互设计、可用性测试等。

3.掌握提示工程的工具和软件：掌握提示工程的工具和软件，如Axure、Sketch、InVision等。

4.实践提示工程的项目：通过实践提示工程的项目，如设计一个网站、一个应用程序或一个产品，来提高自己的技能和经验。

5.参加提示工程的培训和课程：参加提示工程的培训和课程，如在线课程、研讨会、工作坊等，来学习最新的技术和趋势。

希望这些建议能够帮助你开始学习提示工程。
user:U
非常感谢你的建议！那么，你能否推荐一些学习提示工程的资源或者课程呢？
user:A
当然，以下是一些学习提示工程的资源和课程：

1. Coursera上的"人机交互设计"课程：这是一门由加州大学圣地亚哥分校开设的在线课程，涵盖了提示工程的基础知识和技术，包括用户研究、信息架构、交互设计等。

2. Udemy上的"提示工程入门"课程：这是一门由提示工程师开设的在线课程，介绍了提示工程的基础知识和技术，包括用户研究、信息架构、交互设计等。

3. Nielsen Norman Group的网站：这是一个提示工程领域的知名机构，提供了大量的提示工程文章、研究报告和培训课程，可以帮助你深入了解提示工程的最新趋势和技术。

4. Medium上的提示工程专栏：这是一个由提示工程师和专家撰写的专栏，提供了大量的提示工程文章和教程，可以帮助你学习提示工程的实践技巧和经验。

5. Axure RP的官方网站：Axure RP是一款常用的提示工程工具，官方网站提供了大量的教程和资源，可以帮助你掌握Axure RP的使用技巧和方法。

希望这些资源和课程能够帮助你开始学习提示工程。


KeyboardInterrupt: Interrupted by user

**中英文提示**

In [29]:
prompt = '怎么学习提示工程？'
response = chat_completion(prompt)
print(response)

学习提示工程需要掌握以下几个步骤：

1. 学习基础知识：了解提示工程的基本概念、原理和应用场景，掌握相关的技术和工具。

2. 实践项目：通过实践项目来巩固所学知识，提高实践能力和经验。

3. 学习案例：学习提示工程的成功案例，了解其实现过程和经验，从中汲取灵感和启示。

4. 参加培训课程：参加相关的培训课程，学习专业的知识和技能，提高自己的水平。

5. 交流分享：与同行业的人交流分享经验和技巧，了解行业动态和趋势，不断提升自己的能力和竞争力。


In [30]:
prompt = 'How to learn prompt engineering?'
response = chat_completion(prompt)
print(response)

As an AI language model, I don't have personal experience in learning prompt engineering, but here are some steps that can help you learn prompt engineering:

1. Understand the basics of natural language processing (NLP) and machine learning (ML) concepts.

2. Learn about the different types of language models, such as GPT-3, BERT, and T5.

3. Familiarize yourself with the OpenAI API and its documentation.

4. Practice writing prompts and generating responses using the OpenAI API.

5. Join online communities and forums related to prompt engineering to learn from experts and ask questions.

6. Read research papers and articles on prompt engineering to stay up-to-date with the latest developments.

7. Experiment with different techniques and approaches to prompt engineering to find what works best for you.

8. Consider taking online courses or attending workshops on NLP and ML to deepen your understanding of the field.

9. Practice, practice, practice! The more you work with prompt engin

In [31]:
prompt = 'How to learn prompt engineering? Think in English and reply in Chinese.'
response = chat_completion(prompt)
print(response)

如何学习提示工程？要用英语思考并用中文回答。

学习提示工程需要掌握计算机编程和软件开发的基础知识，同时需要了解人工智能、机器学习和自然语言处理等相关技术。以下是一些学习提示工程的建议：

1. 学习编程语言：学习编程语言是学习提示工程的基础。建议学习Python、Java、C++等编程语言。

2. 学习机器学习：机器学习是提示工程的核心技术之一。建议学习机器学习的基本概念、算法和工具，如线性回归、决策树、神经网络等。

3. 学习自然语言处理：自然语言处理是提示工程中的重要技术之一。建议学习自然语言处理的基本概念、技术和工具，如分词、词性标注、命名实体识别等。

4. 学习数据结构和算法：数据结构和算法是编程的基础，也是提示工程中的重要技术之一。建议学习数据结构和算法的基本概念和实现方法。

5. 实践项目：学习提示工程需要实践项目，建议参加相关的比赛和项目，如Kaggle、AI Challenger等。

总之，学习提示工程需要掌握多种技术和工具，需要不断学习和实践。


## 2.2 情景学习

一、在提问之前给出事例，可以激发模型能力，获得更准确的答案


In [32]:
prompt=f'''请写一首五言诗：'''
print(chat_completion(prompt))

春风吹绿江南岸，
花开如锦满山坡。
鸟语声声唤归客，
人间美景胜仙境。


In [33]:
#增加事例，事例中给出了一首五言诗的上下文，模型可以根据上下文准确生成一首五言诗
prompt_1=f'''
春眠不觉晓，处处闻啼鸟。
夜来风雨声，花落知多少。
请写一首五言诗：
'''
print(chat_completion(prompt_1))

春梦未醒时，
鸟声传遍处。
夜雨悄悄落，
花落几多愁。


二、给出与问题更相关的事例，有助于模型正确回答问题

In [34]:
prompt=f'''问题：为什么爸妈结婚没有邀请我参加婚礼？ 答案：'''
print(chat_completion(prompt))

可能是因为他们认为你还太小，或者他们想让婚礼更加简单和私密。也有可能是因为他们认为你可能会分散他们的注意力，或者他们认为你可能会感到无聊或不舒服。无论原因是什么，你应该尊重他们的决定，并祝福他们的婚姻幸福美满。


In [35]:
#增加问题无关的事例，可能无法生成正确的答案
prompt_1=f'''
问题：现任美国总统是谁？ 答案：拜登
问题：为什么爸妈结婚没有邀请我参加婚礼？ 答案：
'''
print(chat_completion(prompt_1))

我作为AI语言模型无法得知具体原因，可能是因为婚礼规模有限、时间紧迫、预算有限或者其他原因。建议与父母沟通，了解具体情况。


In [38]:
#增加与问题相关的事例，有助于生成正确的答案
prompt_2=f'''
问题：为什么我没有参加父母的婚礼？答案：因为父母结婚的时候，我还没有出生，所以你无法参加父母的婚礼。
问题：为什么爸妈结婚没有邀请我参加婚礼？ 答案：
'''
print(chat_completion(prompt_2))

同样的原因，因为你还没有出生，所以父母当时无法邀请你参加婚礼。


三、事例的数量，顺序和多样性等其他多种因素可能会影响最终结果

In [39]:
#错误回答
prompt=f'''问题：一只猫和一只鸡一共有几只脚？ 答案：'''
print(chat_completion(prompt))

4只脚（猫有4只脚，鸡有2只脚）


In [40]:
#增加一个更复杂的事例，并不能生成正确的答案
prompt_1=f'''
问题：两只猫和三只鸡一共有几只脚？ 答案：十四只
问题：一只猫和一只鸡一共有几只脚？ 答案：'''
print(chat_completion(prompt_1))

四只


In [41]:
#增加多个，也无法正确的答案
prompt_2=f'''
问题：两只猫和三只鸡一共有几只脚？ 答案：十四只
问题：三只猫和两只鸡一共有几只脚？ 答案：十六只
问题：一只猫和一只鸡一共有几只脚？ 答案：'''
print(chat_completion(prompt_2))

四只


In [42]:
#增加一个简单事例，可以生成正确的答案
prompt_3=f'''
问题：一只猫有几只脚？ 答案：四只
问题：一只猫和一只鸡一共有几只脚？ 答案：'''
print(chat_completion(prompt_3))

六只


## 2.3 思维链

**通过思维链引导模型生成中间过程，模仿人的思维过程，可以激活模型已有的推理能力。**
（注：ChatGPT已经通过带有CoT的指令进行过微调）

In [44]:
qst = '自助餐厅有23个苹果。如果他们用20个做午饭，再买6个，他们有多少个苹果？'
print('InstructGPT (无CoT):\nA:', completion(qst))
print('-'*40)
print('ChatGPT (有CoT):\nA:', chat_completion(qst))

InstructGPT (无CoT):
A: 

29个苹果。
----------------------------------------
ChatGPT (有CoT):
A: 他们有29个苹果。 

原来有23个，用了20个后还剩3个。再买6个，就是3+6=9个。所以他们现在有23+9=29个苹果。


**创建情景学习的少样本示例**

In [45]:
demo_list = ['罗杰有5个网球。他又买了两罐网球。每个罐子里有3个网球，他现在有多少网球？', '一件长袍需要2匹蓝色纤维和一半的白色纤维。总共需要多少匹？', '克莱尔每天早上做一个3个鸡蛋的煎蛋饼作为早餐。4周后她会吃多少个鸡蛋？', '比利卖DVD。他星期二有8个顾客。他的前3个顾客每人买一张DVD。接下来的2个顾客每人购买2张DVD。他的最后3个顾客没有买DVD。比利星期二卖了多少DVD？', '约翰照顾10只狗。每只狗每天需要.5个小时来散步和处理它们的事务。他每周花多少小时照顾狗？']
a1_list = ['答案是11.', '答案是3.', '答案是7.', '答案是7.', '答案是35.']
a2_list = ['罗杰有5个网球。每个罐子有3个网球，两罐就是2*3=6个网球。5+6=11.答案是11.', '需要2/2=1匹白色纤维，织物总量为2+1=3匹织物。答案是3.', '克莱尔每天吃3个鸡蛋，一周有7天，所以她每周吃3*7=21个鸡蛋。4周之后，她会吃4*21=84个鸡蛋。1打有12个鸡蛋，她会吃掉84个鸡蛋，所以这是84/12=7打鸡蛋。答案是7.', '他的前3位顾客买了3*1=3张DVD。\n接下来的2位顾客买2*2=4张DVD。他总共卖了3+4+0=7张DVD。答案是7.', '他每天花10*.5=5个小时。这意味着他每周花5*7=35个小时。答案是35.']

d1 = ''.join(['Question:{}\nAnaswer:{}\n\n'.format(demo_list[i], a1_list[i]) for i in range(len(demo_list))])
d2 = ''.join(['Question:{}\nAnaswer:{}\n\n'.format(demo_list[i], a2_list[i]) for i in range(len(demo_list))])

In [46]:
print('无CoT的少样本示例（5-shot）:')
print('-'*40)
print(d1)

无CoT的少样本示例（5-shot）:
----------------------------------------
Question:罗杰有5个网球。他又买了两罐网球。每个罐子里有3个网球，他现在有多少网球？
Anaswer:答案是11.

Question:一件长袍需要2匹蓝色纤维和一半的白色纤维。总共需要多少匹？
Anaswer:答案是3.

Question:克莱尔每天早上做一个3个鸡蛋的煎蛋饼作为早餐。4周后她会吃多少个鸡蛋？
Anaswer:答案是7.

Question:比利卖DVD。他星期二有8个顾客。他的前3个顾客每人买一张DVD。接下来的2个顾客每人购买2张DVD。他的最后3个顾客没有买DVD。比利星期二卖了多少DVD？
Anaswer:答案是7.

Question:约翰照顾10只狗。每只狗每天需要.5个小时来散步和处理它们的事务。他每周花多少小时照顾狗？
Anaswer:答案是35.




In [47]:
print('结合了CoT的少样本示例（5-shot）:')
print('-'*40)
print(d2)

结合了CoT的少样本示例（5-shot）:
----------------------------------------
Question:罗杰有5个网球。他又买了两罐网球。每个罐子里有3个网球，他现在有多少网球？
Anaswer:罗杰有5个网球。每个罐子有3个网球，两罐就是2*3=6个网球。5+6=11.答案是11.

Question:一件长袍需要2匹蓝色纤维和一半的白色纤维。总共需要多少匹？
Anaswer:需要2/2=1匹白色纤维，织物总量为2+1=3匹织物。答案是3.

Question:克莱尔每天早上做一个3个鸡蛋的煎蛋饼作为早餐。4周后她会吃多少个鸡蛋？
Anaswer:克莱尔每天吃3个鸡蛋，一周有7天，所以她每周吃3*7=21个鸡蛋。4周之后，她会吃4*21=84个鸡蛋。1打有12个鸡蛋，她会吃掉84个鸡蛋，所以这是84/12=7打鸡蛋。答案是7.

Question:比利卖DVD。他星期二有8个顾客。他的前3个顾客每人买一张DVD。接下来的2个顾客每人购买2张DVD。他的最后3个顾客没有买DVD。比利星期二卖了多少DVD？
Anaswer:他的前3位顾客买了3*1=3张DVD。
接下来的2位顾客买2*2=4张DVD。他总共卖了3+4+0=7张DVD。答案是7.

Question:约翰照顾10只狗。每只狗每天需要.5个小时来散步和处理它们的事务。他每周花多少小时照顾狗？
Anaswer:他每天花10*.5=5个小时。这意味着他每周花5*7=35个小时。答案是35.




**仅基于ICL的LLM在多步推理上表现不佳，但可以通过CoT激活、增强逻辑推理能力。**

In [48]:
qst = 'Jill的教学工资为每小时20美元，担任啦啦队教练的工资为30美元。如果她一年工作50周，每周35小时当老师，每周15小时当教练，她的年薪是多少？'
print('ChatGPT (only ICL):\nA:', chat_completion(d1+qst+'\nA:'))

ChatGPT (only ICL):
A: 答案是$20\times35\times50+30\times15\times50=52500+22500=75000$美元。


In [49]:
qst = 'Jill的教学工资为每小时20美元，担任啦啦队教练的工资为30美元。如果她一年工作50周，每周35小时当老师，每周15小时当教练，她的年薪是多少？'
print('ChatGPT (CoT+ICL):\nA:', chat_completion(d2+qst+'\nA:'))

ChatGPT (CoT+ICL):
A: 她每周作为老师的工资为20*35=700美元。
她每周作为教练的工资为30*15=450美元。
她每周总共的工资为700+450=1150美元。
她一年总共工作50周，所以她的年薪为1150*50=57,500美元。答案是57,500。


# 2. 专业知识应用

## 2.1 专业领域

**数学题目测试**

In [50]:
qst = '简特的鸭子每天下16个蛋。她每天早上吃掉3个蛋作为早餐，并且每天用4个蛋烤松饼给她的朋友们。她每天将剩下的鸭蛋以每个2美元的价格在农民市场上出售。那么她每天在农民市场上能赚多少钱呢？'
print(qst)
print('-'*40)
print(chat_completion(qst))

简特的鸭子每天下16个蛋。她每天早上吃掉3个蛋作为早餐，并且每天用4个蛋烤松饼给她的朋友们。她每天将剩下的鸭蛋以每个2美元的价格在农民市场上出售。那么她每天在农民市场上能赚多少钱呢？
----------------------------------------
每天下16个蛋，吃掉3个，烤松饼用4个，剩下的是16-3-4=9个蛋。
每个蛋以2美元的价格出售，所以9个蛋可以赚9*2=18美元。
因此，简特每天在农民市场上能赚18美元。


**物理知识测试**

In [51]:
qst = '以下做法可以增加铁磁体磁性的是：\nA. 增大电流。\nB. 改变电流方向。\nC. 减少线圈匝数。\nD. 改变绕线方向。'
print(qst)
print('-'*40)
print(chat_completion(qst+'think it step by step and tell me the answer'))

以下做法可以增加铁磁体磁性的是：
A. 增大电流。
B. 改变电流方向。
C. 减少线圈匝数。
D. 改变绕线方向。
----------------------------------------
A. 增大电流。增大电流可以增加铁磁体的磁场强度，从而增加磁性。


In [52]:
qst = '庆元旦晚会上小明为同学们展示了一个物理实验：先将空矿泉水瓶（有少许残余的水）的盖子拧紧，双手用力将瓶子两端向相反方向不断地拧，使瓶子严重变形，再将瓶盖子适当拧松，突然“砰”的一声响，瓶子中的空气将瓶盖高速弹射出去，这时瓶口还产生了明显的白雾．\n（1）试分析说明上述实验过程中，能量的转化过程．\n（2）解释白雾是如何产生的．'
print(qst)
print('-'*40)
print(chat_completion(qst))

庆元旦晚会上小明为同学们展示了一个物理实验：先将空矿泉水瓶（有少许残余的水）的盖子拧紧，双手用力将瓶子两端向相反方向不断地拧，使瓶子严重变形，再将瓶盖子适当拧松，突然“砰”的一声响，瓶子中的空气将瓶盖高速弹射出去，这时瓶口还产生了明显的白雾．
（1）试分析说明上述实验过程中，能量的转化过程．
（2）解释白雾是如何产生的．
----------------------------------------
(1) 在实验过程中，小明用力将瓶子两端向相反方向拧，使瓶子严重变形，这时瓶子内部的空气被压缩，压缩过程中瓶子内部的气体分子运动加剧，产生了热能。当瓶盖被适当拧松后，瓶子内部的压力突然释放，瓶盖高速弹射出去，瓶子内部的气体也随之迅速膨胀，产生了动能。因此，实验过程中能量的转化包括热能转化为动能和弹性势能，以及动能和弹性势能转化为瓶盖的动能和瓶口的热能。

(2) 白雾是由于瓶子内部的空气在压缩过程中被压缩成高压状态，当瓶盖被适当拧松后，瓶子内部的气体突然膨胀，温度降低，水蒸气凝结成小水滴，形成了白雾。


**化学知识测试**

In [53]:
qst='防冻剂是一种能在低温下防止物料中水分结冰的物质。下列可用于防冻剂的物质中，含有氮元素的有机物是\nA. 甲醇 [CH3OH], B. 氯化钙 [CaCI2], C. 尿素 [CO(NH2)2], D. 亚硝酸钠 [NaNO2]'
print(qst)
print('-'*40)
print(chat_completion(qst+'choose the answer and explain'))

防冻剂是一种能在低温下防止物料中水分结冰的物质。下列可用于防冻剂的物质中，含有氮元素的有机物是
A. 甲醇 [CH3OH], B. 氯化钙 [CaCI2], C. 尿素 [CO(NH2)2], D. 亚硝酸钠 [NaNO2]
----------------------------------------
C. 尿素 [CO(NH2)2]。尿素是一种含有氮元素的有机化合物，可以作为防冻剂使用。甲醇虽然可以降低物料的冰点，但是不含氮元素，不是有机物。氯化钙和亚硝酸钠也不含氮元素，不是有机物。


In [54]:
qst = '实验中铜和浓硫酸共热制取二氧化硫，反应的化学方程式是'
print(qst)
print('-'*40)
print(chat_completion(qst))

实验中铜和浓硫酸共热制取二氧化硫，反应的化学方程式是
----------------------------------------
Cu + 2H2SO4 → CuSO4 + SO2 + 2H2O


## 2.2 编程助手

In [55]:
text = f"""
编写一个扫雷游戏
"""
# t #
prompt = f"""
我希望你担任一个精通python的编译器，可以根据需求或者要求，并且输出对应的示例代码，
要求是：'''{text}'''
"""
response = chat_completion(prompt)
print(response)


好的，以下是一个简单的扫雷游戏的示例代码：

```python
import random

class Minesweeper:
    def __init__(self, rows, cols, num_mines):
        self.rows = rows
        self.cols = cols
        self.num_mines = num_mines
        self.board = [[0 for _ in range(cols)] for _ in range(rows)]
        self.mines = set()
        self.game_over = False

        # randomly place mines
        while len(self.mines) < num_mines:
            row = random.randint(0, rows-1)
            col = random.randint(0, cols-1)
            self.mines.add((row, col))
            self.board[row][col] = -1

        # calculate numbers for non-mine cells
        for row in range(rows):
            for col in range(cols):
                if self.board[row][col] == -1:
                    continue
                for r in range(max(0, row-1), min(rows, row+2)):
                    for c in range(max(0, col-1), min(cols, col+2)):
                        if self.board[r][c] == -1:
                            self.board[row][col] += 1

  

## 2.3 语言学习助手

In [ ]:
chatgpt = Conversation('')
while True:
    q = input('user:')
    print("\033[1;32;32mAI:{}\033[0m".format(chatgpt.ask(q)))

## 2.4 科研助手

In [56]:
text = f"""
在论文中插入图片，请你随机生成五组数据并分别用python绘制成小提琴图，箱线图，三维散点图，折线图
"""
# t #
prompt = f"""
我希望你担任一个科研助手，可以实现各种科研和学术写作的需求，并且需要符合一般的论文写作风格和要求，
以下是你需要完成的任务：'''{text}'''
"""
response = chat_completion(prompt)
print(response)

好的，我会尽力完成这些任务。以下是我为您准备的代码和图表。

## 1. 小提琴图

```python
import numpy as np
import matplotlib.pyplot as plt

# 随机生成数据
data = [np.random.normal(0, std, 100) for std in range(1, 6)]

# 绘制小提琴图
fig, ax = plt.subplots()
ax.violinplot(data, showmeans=False, showmedians=True)
ax.set_title('Violin plot')
ax.set_xlabel('Data')
ax.set_ylabel('Frequency')
plt.show()
```

![小提琴图](https://i.imgur.com/5JZJZJL.png)

## 2. 箱线图

```python
import numpy as np
import matplotlib.pyplot as plt

# 随机生成数据
data = [np.random.normal(0, std, 100) for std in range(1, 6)]

# 绘制箱线图
fig, ax = plt.subplots()
ax.boxplot(data)
ax.set_title('Box plot')
ax.set_xlabel('Data')
ax.set_ylabel('Value')
plt.show()
```

![箱线图](https://i.imgur.com/5JZJZJL.png)

## 3. 三维散点图

```python
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 随机生成数据
x = np.random.normal(0, 1, 100)
y = np.random.normal(0, 1, 100)
z = np.random.normal(0, 1, 100)

# 绘制三维散点图
fig = plt.figure()
ax = fig.add_subplot(111, p